# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [1]:
import pandas as pd

In [2]:
from tensorflow.keras.layers.experimental.preprocessing import IntegerLookup
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from tensorflow.keras.layers.experimental.preprocessing import StringLookup
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers

In [3]:
df = pd.read_csv('tic-tac-toe.csv')
df.head()

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,x,x,x,x,o,o,x,o,o,True
1,x,x,x,x,o,o,o,x,o,True
2,x,x,x,x,o,o,o,o,x,True
3,x,x,x,x,o,o,o,b,b,True
4,x,x,x,x,o,o,b,o,b,True


In [4]:
df['class'].value_counts()

True     626
False    332
Name: class, dtype: int64

In [5]:
df.TL.value_counts()

x    418
o    335
b    205
Name: TL, dtype: int64

In [25]:
encoded_df = df.copy()
#encoded_df.TL = pd.factorize(df.TL)[0]
#encoded_df.TM = pd.factorize(df.TM)[0]
#encoded_df.TR = pd.factorize(df.TR)[0]
#encoded_df.ML = pd.factorize(df.ML)[0]
#encoded_df.MM = pd.factorize(df.MM)[0]
#encoded_df.MR = pd.factorize(df.MR)[0]
#encoded_df.BL = pd.factorize(df.BL)[0]
#encoded_df.BM = pd.factorize(df.BM)[0]
#encoded_df.BR = pd.factorize(df.BR)[0]
encoded_df['class'] = pd.factorize(df['class'])[0]

In [19]:
df.columns

Index(['TL', 'TM', 'TR', 'ML', 'MM', 'MR', 'BL', 'BM', 'BR', 'class'], dtype='object')

In [30]:
encoded_df = pd.get_dummies(df, columns=['TL', 'TM', 'TR', 'ML', 'MM', 'MR', 'BL', 'BM', 'BR'], drop_first=True)

In [31]:
encoded_df['class'] = pd.factorize(df['class'])[0]
encoded_df

,class,TL_o,TL_x,TM_o,TM_x,TR_o,TR_x,ML_o,ML_x,MM_o,MM_x,MR_o,MR_x,BL_o,BL_x,BM_o,BM_x,BR_o,BR_x
0,0,0,1,0,1,0,1,0,1,1,0,1,0,0,1,1,0,1,0
1,0,0,1,0,1,0,1,0,1,1,0,1,0,1,0,0,1,1,0
2,0,0,1,0,1,0,1,0,1,1,0,1,0,1,0,1,0,0,1
3,0,0,1,0,1,0,1,0,1,1,0,1,0,1,0,0,0,0,0
4,0,0,1,0,1,0,1,0,1,1,0,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
953,1,1,0,0,1,0,1,0,1,1,0,1,0,1,0,0,1,0,1
954,1,1,0,0,1,1,0,0,1,0,1,1,0,0,1,1,0,0,1
955,1,1,0,0,1,1,0,0,1,1,0,0,1,0,1,1,0,0,1
956,1,1,0,0,1,1,0,1,0,0,1,0,1,0,1,1,0,0,1


## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [32]:
val_dataframe = encoded_df.sample(frac=0.2, random_state=1337)
train_dataframe = encoded_df.drop(val_dataframe.index)

print(
    "Using %d samples for training and %d for validation"
    % (len(train_dataframe), len(val_dataframe))
)

Using 766 samples for training and 192 for validation


In [33]:
def dataframe_to_dataset(dataframe):
    dataframe = dataframe.copy()
    labels = dataframe.pop("class")
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    ds = ds.shuffle(buffer_size=len(dataframe))
    return ds


train_ds = dataframe_to_dataset(train_dataframe)
val_ds = dataframe_to_dataset(val_dataframe)

In [34]:
train_ds = train_ds.batch(32)
val_ds = val_ds.batch(32)

In [35]:
def encode_categorical_feature(feature, name, dataset, is_string):
    lookup_class = StringLookup if is_string else IntegerLookup
    # Create a lookup layer which will turn strings into integer indices
    lookup = lookup_class(output_mode="binary")

    # Prepare a Dataset that only yields our feature
    feature_ds = dataset.map(lambda x, y: x[name])
    feature_ds = feature_ds.map(lambda x: tf.expand_dims(x, -1))

    # Learn the set of possible string values and assign them a fixed integer index
    lookup.adapt(feature_ds)

    # Turn the string input into integer indices
    encoded_feature = lookup(feature)
    return encoded_feature

In [65]:

# Categorical features encoded as integers
TL_o = keras.Input(shape=(1,), name="TL_o", dtype="int64")
TL_x = keras.Input(shape=(1,), name="TL_x", dtype="int64")
TM_o = keras.Input(shape=(1,), name="TM_o", dtype="int64")
TR_o = keras.Input(shape=(1,), name="TR_o", dtype="int64")
ML_o = keras.Input(shape=(1,), name="ML_o", dtype="int64")
MM_o = keras.Input(shape=(1,), name="MM_o", dtype="int64")
MR_o = keras.Input(shape=(1,), name="MR_o", dtype="int64")
BL_o = keras.Input(shape=(1,), name="BL_o", dtype="int64")
BM_o = keras.Input(shape=(1,), name="BM_o", dtype="int64")
BR_o = keras.Input(shape=(1,), name="BR_o", dtype="int64")

TM_x = keras.Input(shape=(1,), name="TM_x", dtype="int64")
TR_x = keras.Input(shape=(1,), name="TR_x", dtype="int64")
ML_x = keras.Input(shape=(1,), name="ML_x", dtype="int64")
MM_x = keras.Input(shape=(1,), name="MM_x", dtype="int64")
MR_x = keras.Input(shape=(1,), name="MR_x", dtype="int64")
BL_x = keras.Input(shape=(1,), name="BL_x", dtype="int64")
BM_x = keras.Input(shape=(1,), name="BM_x", dtype="int64")
BR_x = keras.Input(shape=(1,), name="BR_x", dtype="int64")


all_inputs = [
    TL_o, TM_o, TR_o, ML_o, MM_o, MR_o, BL_o, BM_o, BR_o, TL_x, TM_x, TR_x, ML_x, MM_x, MR_x, BL_x, BM_x, BR_x, 
]

# Integer categorical features
TL_encoded_x = encode_categorical_feature(TL_x, "TL_x", train_ds, False)
TM_encoded_x = encode_categorical_feature(TM_x, "TM_x", train_ds, False)
TR_encoded_x = encode_categorical_feature(TR_x, "TR_x", train_ds, False)
ML_encoded_x = encode_categorical_feature(ML_x, "ML_x", train_ds, False)
MM_encoded_x = encode_categorical_feature(MM_x, "MM_x", train_ds, False)
MR_encoded_x = encode_categorical_feature(MR_x, "MR_x", train_ds, False)
BL_encoded_x = encode_categorical_feature(BL_x, "BL_x", train_ds, False)
BM_encoded_x = encode_categorical_feature(BM_x, "BM_x", train_ds, False)
BR_encoded_x = encode_categorical_feature(BR_x, "BR_x", train_ds, False)

TL_encoded_o = encode_categorical_feature(TL_o, "TL_o", train_ds, False)
TM_encoded_o = encode_categorical_feature(TM_o, "TM_o", train_ds, False)
TR_encoded_o = encode_categorical_feature(TR_o, "TR_o", train_ds, False)
ML_encoded_o = encode_categorical_feature(ML_o, "ML_o", train_ds, False)
MM_encoded_o = encode_categorical_feature(MM_o, "MM_o", train_ds, False)
MR_encoded_o = encode_categorical_feature(MR_o, "MR_o", train_ds, False)
BL_encoded_o = encode_categorical_feature(BL_o, "BL_o", train_ds, False)
BM_encoded_o = encode_categorical_feature(BM_o, "BM_o", train_ds, False)
BR_encoded_o = encode_categorical_feature(BR_o, "BR_o", train_ds, False)

all_features = layers.concatenate(
    [
        TL_encoded_x,
        TM_encoded_x,
        TR_encoded_x,
        ML_encoded_x,
        MM_encoded_x,
        MR_encoded_x,
        BL_encoded_x,
        BM_encoded_x,
        BR_encoded_x,
        TL_encoded_o,
        TM_encoded_o,
        TR_encoded_o,
        ML_encoded_o,
        MM_encoded_o,
        MR_encoded_o,
        BL_encoded_o,
        BM_encoded_o,
        BR_encoded_o,
    ]
)

x = layers.Dense(32, activation="relu")(all_features)
# next line is not necessary but improve results for step 4
#x = layers.Dense(16, activation="relu")(x)
x = layers.Dropout(0.5)(x)

odel = tf.keras.Sequential() 
output = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(all_inputs, output)
model.compile(opt, "binary_crossentropy", metrics=["accuracy"])


In [66]:
model.fit(train_ds, epochs=50, validation_data=val_ds)

Epoch 1/50
24/24 [==============================] - 3s 51ms/step - loss: 0.7218 - accuracy: 0.5509 - val_loss: 0.6254 - val_accuracy: 0.6771
Epoch 2/50
24/24 [==============================] - 0s 13ms/step - loss: 0.5953 - accuracy: 0.6997 - val_loss: 0.5857 - val_accuracy: 0.7240
Epoch 3/50
24/24 [==============================] - 0s 10ms/step - loss: 0.5662 - accuracy: 0.7089 - val_loss: 0.5650 - val_accuracy: 0.7344
Epoch 4/50
24/24 [==============================] - 0s 10ms/step - loss: 0.5548 - accuracy: 0.7193 - val_loss: 0.5497 - val_accuracy: 0.7500
Epoch 5/50
24/24 [==============================] - 0s 13ms/step - loss: 0.5430 - accuracy: 0.7376 - val_loss: 0.5413 - val_accuracy: 0.7396
Epoch 6/50
24/24 [==============================] - 0s 12ms/step - loss: 0.5233 - accuracy: 0.7546 - val_loss: 0.5299 - val_accuracy: 0.7552
Epoch 7/50
24/24 [==============================] - 0s 10ms/step - loss: 0.5250 - accuracy: 0.7376 - val_loss: 0.5200 - val_accuracy: 0.7656
Epoch 8/50
24

## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [42]:
encoded_df.sample()

,class,TL_o,TL_x,TM_o,TM_x,TR_o,TR_x,ML_o,ML_x,MM_o,MM_x,MR_o,MR_x,BL_o,BL_x,BM_o,BM_x,BR_o,BR_x
252,0,0,1,0,0,1,0,0,1,1,0,0,0,0,1,1,0,0,1


In [52]:
sample = {
    "TL_x": 1,
    "TM_x": 1,
    "TR_x": 2,
    "ML_x": 0,
    "MM_x": 0,
    "MR_x": 1,
    "BL_x": 0,
    "BM_x": 1,
    "BR_x": 0,
    "TL_o": 1,
    "TM_o": 1,
    "TR_o": 2,
    "ML_o": 0,
    "MM_o": 0,
    "MR_o": 1,
    "BL_o": 0,
    "BM_o": 1,
    "BR_o": 0,
}

        
input_dict = {name: tf.convert_to_tensor([value]) for name, value in sample.items()}
predictions = model.predict(input_dict)

print(
    "This particular combination had a %.1f percent probability "
    "of existing, as evaluated by our model." % (100 * predictions[0][0],)
)

This particular combination had a 6.9 percent probability of existing, as evaluated by our model.


## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [61]:
model = keras.Model(all_inputs, output)
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(opt, "binary_crossentropy", metrics=["accuracy"])

In [62]:
model.fit(train_ds, epochs=50, validation_data=val_ds)

Epoch 1/50
24/24 [==============================] - 2s 38ms/step - loss: 0.0182 - accuracy: 1.0000 - val_loss: 0.0865 - val_accuracy: 0.9844
Epoch 2/50
24/24 [==============================] - 0s 8ms/step - loss: 0.0221 - accuracy: 0.9948 - val_loss: 0.0697 - val_accuracy: 0.9740
Epoch 3/50
24/24 [==============================] - 0s 14ms/step - loss: 0.0146 - accuracy: 0.9987 - val_loss: 0.0817 - val_accuracy: 0.9844
Epoch 4/50
24/24 [==============================] - 0s 8ms/step - loss: 0.0165 - accuracy: 0.9987 - val_loss: 0.0715 - val_accuracy: 0.9844
Epoch 5/50
24/24 [==============================] - 0s 16ms/step - loss: 0.0196 - accuracy: 0.9974 - val_loss: 0.0698 - val_accuracy: 0.9792
Epoch 6/50
24/24 [==============================] - 0s 14ms/step - loss: 0.0196 - accuracy: 0.9974 - val_loss: 0.0795 - val_accuracy: 0.9844
Epoch 7/50
24/24 [==============================] - 0s 8ms/step - loss: 0.0124 - accuracy: 1.0000 - val_loss: 0.0654 - val_accuracy: 0.9844
Epoch 8/50
24/24

In [56]:
model.evaluate(train_ds)

24/24 [==============================] - 0s 7ms/step - loss: 0.0307 - accuracy: 0.9922


[0.03068537265062332, 0.9921671152114868]

**Which approach(es) did you find helpful to improve your model performance?**

Both